# Creating your first agent

[![Your first agent](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) 

If you're looking to build your first agent using a few simple tools, this is a great place to start. In this cookbook example, we will create and run a simple agent that has access to a few web tools. This can be easily expanded to add more advanced tools and features. 🚀

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[all]'

import nest_asyncio

nest_asyncio.apply()

## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the "openai" agent framework.

### Pick an LLM

Regardless of which agent framework you choose, each framework supports LiteLLM, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use OpenAI's gpt-4.1-nano.

### Pick which tools to use

 In this tutorial, we will provide the agent with access to web searches and web page visits. In later examples we will show more advanced tool usage like Model Context Protocol (MCP) tool use or inter-agent communication using Agent-2-Agent (A2A).

In [8]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

OPENAI_API_KEY found in environment.


In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import search_web, visit_webpage

agent = AnyAgent.create(
    "openai",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(model_id="gpt-4.1-nano", tools=[search_web, visit_webpage]),
)

## Run the Agent

Now we've configured our agent, so it's time to run it! Let's give it a simple task: find 5 trending new TV shows that were released recently.


In [3]:
agent_trace = agent.run("What are 5 tv shows that are trending in 2025?")

────────────────────────────────────────────────────── TOOL ───────────────────────────────────────────────────────

tool_name: search_web

input: {'query': 'Trending TV shows in 2025'}

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ ]8;id=530931;https://editorial.rottentomatoes.com/guide/best-new-tv-series-shows/\Best TV Shows of 2025: New Series to Watch Now - Rotten Tomatoes]8;;\ The latest: May kicks off with Poker Face:     │
│ Season 2, #1 Happy Family USA: Season 1, North of North: Season 1, The Four Seasons: Season 1, and Doctor Who:  │
│ Season 2. You: Season 5, Étoile: Season 1, and The Narrow Road to the Deep North close out the month. Andor     │
│ sticks the landing with a Certified Fresh Season 2. The Wheel of Time continues to grow in stature, now with    │
│ Season 3 the series ... ]8;id=697958;https://deadline.com/lists/2025-tv-shows/\2025 TV Shows: Release Dates For Most Anticipated Series - Deadline]8;;\ 36 Of The Most      │
│ Anticipated New & Returning TV Shows Of 2025. By Dessi Gomez. Dessi Gomez. ... some long-anticipated returning  │
│ shows have new seasons arriving in 2025, ... Trending on Deadline ]8;id=541647;https://www.timeout.com/film/must-see-tv-shows-for-2025-you-cant-miss\23 Best Streaming and TV Shows to See in 2025]8;;\ │
│ ]8;id=541647;https://www.timeout.com/film/must-see-tv-shows-for-2025-you-cant-miss\- Time Out]8;;\ These are the 23 must-see TV shows for 2025 you can't miss. A new 'White Lotus', the end of          │
│ 'Stranger Things', and a knockout boxing drama. Monday 6 January 2025. Share. Copy Link. ]8;id=256015;https://www.empireonline.com/tv/features/best-tv-shows-2025/\87 TV Series You Must ]8;;\ │
│ ]8;id=256015;https://www.empireonline.com/tv/features/best-tv-shows-2025/\See In 2025 (And Where To Stream Them)]8;;\ From Andor to Zero Day, and The Last Of Us to Stranger Things, Empire    │
│ presents your guide to the biggest shows streaming in 2025. Read the full list. ]8;id=282886;https://variety.com/lists/most-anticipated-tv-shows-2025/\The 41 Most Anticipated New and]8;;\ │
│ ]8;id=282886;https://variety.com/lists/most-anticipated-tv-shows-2025/\Returning TV Shows Coming in 2025 - Variety]8;;\ The show was also revived for one season at HGTV in 2020 with host  │
│ Jesse Tyler Ferguson. The new series, produced by Endemol Shine North America and Hello Sunshine, will feature  │
│ the hosts meeting ... ]8;id=505648;https://www.hollywoodreporter.com/lists/most-anticipated-tv-shows-2025/\The 31 Most Anticipated Scripted TV Shows of 2025 - The Hollywood Reporter]8;;\ The            │
│ after-effects of media conglomerate cost-cutting and 2023's labor strikes showed up in noticeable ways for TV   │
│ viewers last year. Networks and streamers made fewer shows — the Peak TV era ... ]8;id=631865;https://www.polygon.com/tv/493214/best-new-tv-2025-release-calendar-schedule-shows-series\The 50 big new TV shows to ]8;;\    │
│ ]8;id=631865;https://www.polygon.com/tv/493214/best-new-tv-2025-release-calendar-schedule-shows-series\watch in 2025 | Polygon]8;;\ The 50 most exciting TV shows for your 2025 watch list. The year is full of             │
│ long-awaited returns and exciting new premieres. by Austen Goslin and Polygon Staff. Jan 1, 2025, 2:00 PM UTC   │
│ ]8;id=882986;https://tvline.com/features/2025-tv-shows-1235385965/\All the New TV Shows to Look Forward to in 2025, List of ... - TVLine]8;;\ TVLine presents a comprehensive list of   │
│ every new TV show premiering across broadcast, cable and streaming in 2025. ... Trending Shows. 9-1-1; 1923;    │
│ ... And that's only the first two months of ... ]8;id=427516;https://www.imdb.com/list/ls593164948\new shows/series to watch - 2025 - IMDb]8;;\ With an exciting lineup │
│ of fresh stories, compelling characters, and intriguing concepts, this list is dedicated to the latest          │
│ must-watch series of 2025. Covering a mix of gripping crime dramas, heartfelt family narratives, thrilling      │
│ procedural spin offs, and nostalgic revivals, these new releases bring something for every type

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ ]8;id=974272;https://editorial.rottentomatoes.com/guide/best-new-tv-series-shows/\Best TV Shows of 2025: New Series to Watch Now - Rotten Tomatoes]8;;\ The latest: May kicks off with Poker Face:     │
│ Season 2, #1 Happy Family USA: Season 1, North of North: Season 1, The Four Seasons: Season 1, and Doctor Who:  │
│ Season 2. You: Season 5, Étoile: Season 1, and The Narrow Road to the Deep North close out the month. Andor     │
│ sticks the landing with a Certified Fresh Season 2. The Wheel of Time continues to grow in stature, now with    │
│ Season 3 the series ... ]8;id=533919;https://www.timeout.com/film/must-see-tv-shows-for-2025-you-cant-miss\23 Best Streaming and TV Shows to See in 2025 - Time Out]8;;\ Netflix blew the cobwebs off   │
│ The Addams Family property in 2022, turning it into a gothy YA mystery series focused on the franchise's best   │
│ character, and scored a massive hit. Season 2 won't ... ]8;id=455147;https://time.com/7285497/most-anticipated-tv-shows-summer-2025/\The 36 Most Anticipated TV Shows of Summer 2025 | TIME]8;;\  │
│ Alexander Skarsgård stars as the content-loving humanoid, and is supported by a cast of dramedy heavy-hitters   │
│ like David Dastmalchian, Noma Dumezweni, Sabrina Wu, John Cho and Jack McBrayer ... ]8;id=944680;https://www.rd.com/article/most-anticipated-tv-shows-2025/\The 10 Best TV Shows of ]8;;\    │
│ ]8;id=944680;https://www.rd.com/article/most-anticipated-tv-shows-2025/\2025 to Put on Your Must-Watch List]8;;\ Find out the buzziest TV shows of 2025, from Severance to Yellowjackets,    │
│ and when and where to watch them. Reader's Digest rounds up the year's must-see TV shows, along with spoilers,  │
│ cast and trailers. ]8;id=686913;https://www.empireonline.com/tv/features/best-tv-shows-2025/\87 TV Series You Must See In 2025 (And Where To Stream Them)]8;;\ A comprehensive list of the     │
│ best TV shows coming out in 2025, from Severance to Stranger Things, with streaming platforms and release       │
│ dates. Find out what to watch in the next 11 months of TV, from returning favourites to new originals. ]8;id=882750;https://www.vanityfair.com/hollywood/story/best-tv-shows-2025\The Best]8;;\ │
│ ]8;id=882750;https://www.vanityfair.com/hollywood/story/best-tv-shows-2025\TV Shows of 2025, So Far - Vanity Fair]8;;\ We're only three months into 2025, but this young year is already        │
│ chock-full of incredible television. The best TV shows of 2025 (so far) can be found on streaming, premium      │
│ cable, and even (even ... ]8;id=838433;https://www.imdb.com/list/ls543358543/\The Most Anticipated TV Shows of 2025 (/2026) - IMDb]8;;\ List your movie, TV & celebrity  │
│ picks. An architect works to build up the 1893 World's Fair, while the serial killer H. H. Holmes uses the fair │
│ to attract and kill women ... ]8;id=637326;https://www.vulture.com/article/best-tv-shows-new-series-2025.html\The Best TV Shows and New Series of 2025 (So Far) - Vulture]8;;\ From ballet comedy to │
│ Star Wars prequel, from social experiment to war drama, Vulture reviews the standout series of 2025. See the    │
│ list of six shows and their premises, critics' opinions, and links to more coverage. ]8;id=675812;https://www.hollywoodreporter.com/lists/most-anticipated-tv-shows-2025/\The 31 Most Anticipated ]8;;\   │
│ ]8;id=675812;https://www.hollywoodreporter.com/lists/most-anticipated-tv-shows-2025/\Scripted TV Shows of 2025 - The Hollywood Reporter]8;;\ The 31 Most Anticipated Scripted TV Shows of 2025            │
│ Long-delayed returning favorites and several intriguing new titles are on tap for the coming year. By Rick      │
│ Porter , Nicole Fell , James Hibberd ... ]8;id=564637;https://www.esquire.com/uk/culture/tv/a63332100/best-tv-shows-2025/\The Best TV Shows of 2025: Release Dates and Trailers - Esquire]8;;\

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

─────────────────────────────────────────────────────── LLM ───────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ Here are 5 trending TV shows in 2025:                                                                           │
│                                                                                                                 │
│  1 Poker Face: Season 2 - A new season of the popular mystery series.                                           │
│  2 Happy Family USA: Season 1 - A family-centered comedy-drama.                                                 │
│  3 North of North: Season 1 - An intriguing new series with a focus on northern territories.                    │
│  4 The Four Seasons: Season 1 - A show exploring stories across different seasons.                              │
│  5 Doctor Who: Season 2 - The latest season of the iconic sci-fi series.                                        │
│                                                                                                                 │
│ Would you like more details about any of these shows?                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [7]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.usage}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")

Here are 5 trending TV shows in 2025:

1. **Poker Face: Season 2** - A new season of the popular mystery series.
2. **Happy Family USA: Season 1** - A family-centered comedy-drama.
3. **North of North: Season 1** - An intriguing new series with a focus on northern territories.
4. **The Four Seasons: Season 1** - A show exploring stories across different seasons.
5. **Doctor Who: Season 2** - The latest season of the iconic sci-fi series.

Would you like more details about any of these shows?
Duration: 5.87 seconds
Usage: total_token_count_prompt=2267 total_token_count_completion=183 total_tokens=2450
Cost (USD): 0.000227
